<a href="https://colab.research.google.com/github/imabari/covid19-data/blob/master/mhlw_ocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# データをダウンロード
!wget https://www.mhlw.go.jp/content/10906000/000668093.pdf -O data.pdf
# !wget https://www.mhlw.go.jp/content/10906000/000668309.pdf -O data.pdf

--2020-09-06 02:20:18--  https://www.mhlw.go.jp/content/10906000/000668093.pdf
Resolving www.mhlw.go.jp (www.mhlw.go.jp)... 23.78.64.19, 2600:1408:5400:48d::3194, 2600:1408:5400:4a8::3194
Connecting to www.mhlw.go.jp (www.mhlw.go.jp)|23.78.64.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 453107 (442K) [application/pdf]
Saving to: ‘data.pdf’

data.pdf            100%[===================>] 442.49K  --.-KB/s    in 0.09s   

2020-09-06 02:20:19 (4.76 MB/s) - ‘data.pdf’ saved [453107/453107]



In [2]:
# PDFBOXをダウンロード
!wget https://www-eu.apache.org/dist/pdfbox/2.0.21/pdfbox-app-2.0.21.jar -O pdfbox-app.jar

--2020-09-06 02:20:19--  https://www-eu.apache.org/dist/pdfbox/2.0.21/pdfbox-app-2.0.21.jar
Resolving www-eu.apache.org (www-eu.apache.org)... 95.216.24.32, 2a01:4f9:2a:185f::2
Connecting to www-eu.apache.org (www-eu.apache.org)|95.216.24.32|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://downloads.apache.org/pdfbox/2.0.21/pdfbox-app-2.0.21.jar [following]
--2020-09-06 02:20:19--  https://downloads.apache.org/pdfbox/2.0.21/pdfbox-app-2.0.21.jar
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 2a01:4f8:10a:201a::2
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9897381 (9.4M) [application/java-archive]
Saving to: ‘pdfbox-app.jar’

pdfbox-app.jar      100%[===================>]   9.44M  9.37MB/s    in 1.0s    

2020-09-06 02:20:21 (9.37 MB/s) - ‘pdfbox-app.jar’ saved [9897381/9897381]



In [3]:
# PDFから画像に変換
!java -jar pdfbox-app.jar PDFToImage -imageType png -dpi 300 data.pdf

In [4]:
# !java -jar pdfbox-app.jar ExtractImages data.pdf

In [5]:
!add-apt-repository ppa:alex-p/tesseract-ocr -y

!apt update

!apt install tesseract-ocr
!apt install libtesseract-dev

!tesseract -v

!apt install tesseract-ocr-jpn  tesseract-ocr-jpn-vert
!apt install tesseract-ocr-script-jpan tesseract-ocr-script-jpan-vert

!tesseract --list-langs
!pip install pytesseract

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/alex-p/tesseract-ocr/ubuntu bionic InRelease [15.4 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:12 https://cloud.r-p

In [6]:
import re

import cv2
import numpy as np
import pytesseract

In [7]:
from google.colab.patches import cv2_imshow

参考：[OpenCVでの表のセルの認識方法](https://teratail.com/questions/151317)

In [8]:
img = cv2.imread("data1.png")

# BGR -> グレースケール
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

gray[gray > 220] = 255
cv2.imwrite("gray.png", gray)

# エッジ抽出 (Canny)
edges = cv2.Canny(gray, 1, 100, apertureSize=3)
cv2.imwrite("edges.png", edges)

# 膨張処理
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
edges = cv2.dilate(edges, kernel)

# 輪郭抽出
contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

In [9]:
rects = []

for cnt, hrchy in zip(contours, hierarchy[0]):

    if hrchy[3] == -1:
        continue  # ルートノードは除く

    # 面積範囲
    if 6000 < cv2.contourArea(cnt) < 12000:

        # 輪郭を囲む長方形を計算する。
        rect = cv2.minAreaRect(cnt)
        rect_points = cv2.boxPoints(rect).astype(int)

        # 座標
        x_max, y_max = np.amax(rect_points.T, axis=1)
        x_min, y_min = np.amin(rect_points.T, axis=1)

        # 各座標ごとに切り出し
        dst = gray[y_min + 1 : y_max - 1, x_min + 5 : x_max]

        # OCR用に1.2倍に拡大
        dst2 = cv2.resize(dst, None, fx=1.2, fy=1.2)
        # cv2.imwrite(f"gray{i:03}.png", dst2)

        txt = (
            pytesseract.image_to_string(dst2, lang="jpn", config="--psm 6")
            .replace(",", "")
            .replace(".", "")
            .replace(" ", "")
            .strip()
        )

        if txt:
            rects.append([y_min, y_max , x_min, x_max, txt])

In [10]:
import pandas as pd

In [11]:
df = pd.DataFrame(rects, columns=["y1", "y2", "x1", "x2", "text"])

In [12]:
# X軸のずれ調整
t1 = set(df["x1"].unique()) & set(df["x1"].unique() + 1)

for i in t1:
    df["x1"] = df["x1"].replace(i, i - 1)

In [13]:
# Y軸のずれ調整
t2 = set(df["y2"].unique()) & set(df["y2"].unique() + 1)

for i in t2:
    df["y2"] = df["y2"].replace(i, i - 1)

In [14]:
pv = df.pivot_table(index="y2", columns="x1", values="text", aggfunc=lambda x: " ".join(str(v) for v in x))

In [15]:
pv.dropna(thresh=3, inplace=True)

In [16]:
pv.dropna(how="all", axis=1, inplace=True)

In [17]:
pv.set_axis(
    ["都道府県名", "陽性者数", "PCR検査実施人数", "入院", "重症", "退院", "死亡", "確認中"], axis=1, inplace=True
)

In [18]:
pv.set_index("都道府県名", inplace=True)

In [19]:
pv.set_axis(
    [
        "北海道",
        "青森県",
        "岩手県",
        "宮城県",
        "秋田県",
        "山形県",
        "福島県",
        "茨城県",
        "栃木県",
        "群馬県",
        "埼玉県",
        "千葉県",
        "東京都",
        "神奈川県",
        "新潟県",
        "富山県",
        "石川県",
        "福井県",
        "山梨県",
        "長野県",
        "岐阜県",
        "静岡県",
        "愛知県",
        "三重県",
        "滋賀県",
        "京都府",
        "大阪府",
        "兵庫県",
        "奈良県",
        "和歌山県",
        "鳥取県",
        "島根県",
        "岡山県",
        "広島県",
        "山口県",
        "徳島県",
        "香川県",
        "愛媛県",
        "高知県",
        "福岡県",
        "佐賀県",
        "長崎県",
        "熊本県",
        "大分県",
        "宮崎県",
        "鹿児島県",
        "沖縄県",
        "その他",
        "合計",
    ],
    inplace=True,
)

In [20]:
pv = pv.astype(float).astype("Int64")

In [21]:
pv.dtypes

陽性者数         Int64
PCR検査実施人数    Int64
入院           Int64
重症           Int64
退院           Int64
死亡           Int64
確認中          Int64
dtype: object

In [22]:
pv.to_csv("data.csv")

# 合計確認

In [26]:
 # 合計
 pv.loc["合計"]

陽性者数           69450
PCR検査実施人数    1387330
入院              8363
重症               214
退院             59720
死亡              1329
確認中               88
Name: 合計, dtype: Int64

In [27]:
# 集計
pv.iloc[:-1].sum().astype("Int64")

陽性者数           69450
PCR検査実施人数    1387330
入院              8363
重症               214
退院             59720
死亡              1329
確認中               88
dtype: Int64

In [28]:
pv.iloc[:-1].sum() == pv.loc["合計"]

陽性者数         True
PCR検査実施人数    True
入院           True
重症           True
退院           True
死亡           True
確認中          True
dtype: boolean